In [4]:

FIELD192 = 4787605948707450321761805915146316350821882368518086721537
F = GF(FIELD192)  
R.<x> = PolynomialRing(F)
d=2**20
poly = R.random_element(degree=d)
# print(poly)
# R = NTT_Ring(F, N)
# log_rho_inv = 2
# size = d * (1 << log_rho_inv)
# f_ntt = R.fft(poly)  





In [5]:
###############################################################################
# 1.  Parameters
###############################################################################
n  = 2^20                        # transform length (power of two is simplest)
p  = 998_244_353                 # 2^23·119 + 1   – a classic “NTT-friendly” prime
F  = GF(p)                       # prime finite field 𝔽p

#–– Every NTT needs a primitive n-th root of unity in the field ––#
g     = F(3)                     # 3 is a generator of 𝔽p×
omega = g^((p-1)//n)             # ω = g^{(p−1)/n}
assert omega^n == 1 and all((omega^k != 1) for k in (1..n-1))    # safety check

###############################################################################
# 2.  A reference polynomial we want to transform
###############################################################################
R.<x> = PolynomialRing(F)
poly  = R.random_element(degree=n-1)   # random polynomial of degree < n
coeff = poly.list() + [F(0)]*(n - poly.degree() - 1)  # pad to length n

###############################################################################
# 3.  Cooley–Tukey radix-2 iterative NTT / iNTT
###############################################################################
def bitrev(i, bits):                      # standard bit-reverse helper
    r = 0
    for _ in range(bits):
        r = (r<<1) | (i & 1)
        i >>= 1
    return r

def ntt(a, root):
    """in-place radix-2 NTT (root = ω or ω⁻¹)"""
    N     = len(a)
    logN  = N.nbits() - 1
    # –– bit-reversal permutation ––
    for i in range(N):
        j = bitrev(i, logN)
        if i < j:
            a[i], a[j] = a[j], a[i]

    len_ = 1
    while len_ < N:
        w_m = root^(N // (2*len_))
        for k in range(0, N, 2*len_):
            w = F(1)
            for j in range(len_):
                u = a[k + j]
                v = w * a[k + j + len_]
                a[k + j]           = u + v
                a[k + j + len_]    = u - v
                w *= w_m
        len_ <<= 1
    return a

def intt(a):
    n_inv = F(1) / F(len(a))
    ntt(a, omega^-1)                 # inverse transform is NTT with ω⁻¹
    for i in range(len(a)):          # scale by n⁻¹
        a[i] *= n_inv
    return a

###############################################################################
# 4.  Perform forward and inverse transforms
###############################################################################
A  = ntt(coeff[:], omega)           # forward NTT  (copy because it’s in-place)
a2 = intt(A[:])                     # back to time domain

assert coeff == a2                  # round-trip test
print("NTT / iNTT round-trip successful.")

AttributeError: 'int' object has no attribute 'nbits'